# SCOUT Simulation Vignette

## Overview

This vignette demonstrates the basics of how to use SCOUT. We: 
1. Simulate single-cell lineage trees with gene expression data
2. Fit evolutionary models (BM1, OU1, OUM) to the simulated data
3. Perform model selection and recover known parameters

The simulation framework allows you to test SCOUT's ability to distinguish between different evolutionary processes and accurately estimate parameters.

## What You'll Learn

- How to generate synthetic single-cell lineage trees
- How to simulate gene expression evolving under different evolutionary models
- How to visualize simulated trees and gene expression patterns
- How to run SCOUT's complete workflow on simulated data


It's easiest to run this inside the **examples** directory, overwriting the previous simulation data


## Setup and Library Loading

First, we'll load the required libraries:
- `SCOUT`: Main package for evolutionary modeling
- `ComplexHeatmap`: For visualizing expression patterns
- `ggtree`: For phylogenetic tree visualization

### Installing required libraries

If you don't have the following libraries installed you can do so here:

In [ ]:
# Generally don't run this cell unless you need to
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("ComplexHeatmap")

BiocManager::install("ggtree")

install.packages("future")
install.packages("progressr")
install.packages("future.apply")
install.packages("tidyverse")

### Load the libraries and SCOUT

In [ ]:
# Reload SCOUT to ensure latest version
library(SCOUT)
library(ComplexHeatmap)
library(ggtree)
library(future)
library(progressr)
library(future.apply)
library(tidyverse)

## Theoretical Background: The Ornstein-Uhlenbeck Process

The Ornstein-Uhlenbeck (OU) process models trait evolution with selection toward an optimum:

$$dX(t) = \alpha[\theta - X(t)]dt + \sigma dW(t)$$

Where:
- **α (alpha)**: Selection strength - how strongly the trait is pulled toward the optimum
  - α = 0: No selection (equivalent to Brownian Motion)
  - α > 0: Stronger selection toward optimum
- **θ (theta)**: Optimal trait value - the value toward which selection pulls
- **σ² (sigma²)**: Stochastic motion parameter - represents random drift
- **W(t)**: Wiener process (Brownian motion)

### Models in SCOUT:

1. **BM1** (Brownian Motion): α = 0, constant σ² across tree
2. **OU1** (Single OU): Single α, θ, and σ² for entire tree
3. **OUM** (Multiple Optima OU): Single α and σ², but different θ values for different regimes

## Step 1: Simulate Test Data

We'll generate a synthetic dataset with:
- **30 genes per evolutionary regime** (BM1, OU1, OUM)
- **128 cells** arranged in a lineage tree
- **Multiple parameter combinations** to test SCOUT's ability to discriminate between different evolutionary dynamics

### Simulation Parameters:

- `ngenes`: Number of genes to simulate per regime (30)
- `ncells`: Number of cells in the lineage tree (128)
- `tree`: Cell state tree topology in Newick format. The tree defines the number of evolutionary regimes/states. For example: `'(((t1:0.5,t2:0.5):0.5,(t3:0.5,t4:0.5):0.5):0.5);'` creates 4 states
- `a`: Selection strength (α) values to test: [0.5, 1.0, 1.5]
- `s`: Stochastic parameter (σ) values to test: [0.1, 0.5, 1.0]
- `t0`: Initial theta (optimum) value (3.0)
- `theta_step`: How much theta differs between regimes (1.0)
- `nevfs`: Number of expression variance factors for SymSim pipeline (10)

In [ ]:
# Simulate test data with multiple parameter combinations


sim.res <- simulate_test_data(ngenes = 10, # number of genes to simulat per regime.  
                              ncells = 128, # number of cells to simulate. # genes < # cells 
                              tree ='((t3:1, t4:1):1);', # cell state tree 
                              outdir = './simulate_data', # output directory 
                              out_prefix = 'sim_test', # prefix for output files 
                              a = c(0.5, 1, 1.5), # alpha parameters to simulate
                              s = c(0.1, 0.5, 1), # sigma parameters to simulate 
                              t0 = 3,  # initial theta value for OUwie.sim 
                              theta_step = 1, # for multiple theta values, how offset should the different regimes be. 
                              nevfs = 10) # number of EVFs to generate (for SymSim pipeline)



## Step 2: Explore Simulation Results

The `simulate_test_data()` function returns a nested list containing:

### Main Components:
- `scLT`: Single-cell lineage tree data
  - `tree`: Phylogenetic tree object
  - `observed_counts`: Cell barcode counts
  - `meta`: Metadata with cell IDs and regime annotations
  - `states`: Cell state information
- `simulated_OU`: Ornstein-Uhlenbeck simulated gene expression
  - `counts`: Gene expression count matrices
  - `evfs`: Expression variance factors

Let's examine the structure:

In [ ]:
# Examine the structure of simulation results
names(sim.res)
names(sim.res$simulated_OU)
names(sim.res$scLT)

## Step 3: Visualize Simulated Tree with Cell State Annotations

We'll create a dendrogram showing the lineage tree with cell states color-coded.
This helps us understand the structure of the simulated data and how cells are organized into different regimes.

In [ ]:
# Set plot dimensions for better visualization
options(repr.plot.width = 12, repr.plot.height = 6)

# Extract metadata and create annotation
meta <- sim.res$scLT$meta
annotation <- as.factor(meta$cluster)
names(annotation) <- meta$nodeID

# Create tree plot with state annotations
p <- ggtree(sim.res$scLT$tree) + layout_dendrogram()
gheatmap(p, data.frame(annotation), width = 0.1)

## Step 4: Examine Simulated Gene Expression

The simulated expression data contains genes following different evolutionary models:
- **BM1_1 to BM1_30**: Genes evolving under Brownian Motion
- **OU1_1 to OU1_30**: Genes evolving under single-optimum OU
- **OUM_1 to OUM_30**: Genes evolving under multi-optimum OU

Each set of 30 genes represents different combinations of the α and σ parameters we specified.
The count data simulates realistic single-cell RNA-seq counts.

In [ ]:
# Display the first few rows of simulated gene expression
# Notice the different patterns across BM1, OU1, and OUM genes
head(sim.res$simulated_OU$counts[[1]])

## Next Steps: Running SCOUT on Simulated Data

Now that we have simulated data, the next steps would be:


In [ ]:
# make the output directory if it doesn't exist
dir.create("./scout_output", recursive = TRUE, showWarnings = FALSE)

### 1. Prepare data for SCOUT
inputs <- prepare_data(
    tree_path = "./simulate_data/sim_test_newick.nwk",
    metadata_path = sim.res$simulated_OU$counts[[1]],
    outpath = "./scout_output",
    regimes = c("BM1", "OU1", "OUM"),
    normalize = TRUE,
    smoothing = 10
)


## Now we can fit models to our simulated results

In [ ]:
### 2. Fit models
results <- fitModel(
    inputs,
    cores = 10, # fill this in with the number of cores you can spare on your computer!
    write = TRUE,
    outpath = "./scout_output"
)


In [ ]:

### 3. Calculate fit metrics and perform model selection
results <- get_fit_metrics(results, write = TRUE)
final_results <- processes_results(results)


## Now we can look at the classifications for each gene and the scores:

In [ ]:
final_results$results


### 4. Validate parameter recovery
You'll next want to compare the estimated parameters (α, σ², θ) to the known simulation parameters to assess SCOUT's accuracy

## Summary

This vignette demonstrated:
- ✓ How to generate synthetic single-cell lineage trees
- ✓ How to simulate gene expression under different evolutionary models
- ✓ How to visualize and explore simulated data
- ✓ The structure of SCOUT simulation outputs

The simulated data provides a ground truth for testing SCOUT's ability to:
- Distinguish between BM, OU, and OUM models
- Accurately estimate selection strength (α)
- Identify optimal trait values (θ) across different regimes
- Quantify stochastic variation (σ²)